In [59]:
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

from sklearn.metrics import confusion_matrix, classification_report

import src.data_cleaning as dc

In [2]:
df = dc.create_df('bigml_59c28831336c6604c800002a.csv')

In [3]:
df['churn'].value_counts()

False    2850
True      483
Name: churn, dtype: int64

In [4]:
X = df.drop('churn', axis=1)
y = df['churn']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [6]:
X_train.dtypes

state                      object
account length              int64
area code                   int64
phone number               object
international plan         object
voice mail plan            object
number vmail messages       int64
total day minutes         float64
total day calls             int64
total day charge          float64
total eve minutes         float64
total eve calls             int64
total eve charge          float64
total night minutes       float64
total night calls           int64
total night charge        float64
total intl minutes        float64
total intl calls            int64
total intl charge         float64
customer service calls      int64
dtype: object

In [7]:
X_train.drop(['state', 'phone number', 'international plan', 'voice mail plan'], axis=1, inplace=True)
X_test.drop(['state', 'phone number', 'international plan', 'voice mail plan'], axis=1, inplace=True)

/Users/kyledecember1/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [8]:
ss = StandardScaler()

X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

In [101]:
y_train.value_counts()

False    2141
True      358
Name: churn, dtype: int64

### Dummy Model

In [46]:
dummy = DummyClassifier(random_state=42)

dummy.fit(X_train_scaled, y_train)
preds_dumb = dummy.predict(X_train_scaled)
print(classification_report(y_train, preds_dumb))

              precision    recall  f1-score   support

       False       0.86      0.85      0.86      2141
        True       0.15      0.16      0.16       358

    accuracy                           0.75      2499
   macro avg       0.51      0.51      0.51      2499
weighted avg       0.76      0.75      0.76      2499



### Model 1 : RandomForest

In [88]:
rf1 = RandomForestClassifier(max_depth=20, random_state=42, min_samples_leaf=3, n_estimators=1000)

rf1.fit(X_train_scaled, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=20, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [89]:
rf1.score(X_train_scaled, y_train)

0.9551820728291317

In [90]:
rf1.feature_importances_

array([0.03242416, 0.00800068, 0.04569875, 0.15844243, 0.03342019,
       0.16587861, 0.07834606, 0.03120913, 0.07994629, 0.03964869,
       0.03688218, 0.0392288 , 0.03852835, 0.02547047, 0.03638024,
       0.15049496])

In [91]:
preds_1 = rf1.predict(X_train_scaled)

confusion_matrix(y_train, preds_1)

array([[2141,    0],
       [ 112,  246]])

In [92]:
cross_val_score(rf1, X_train_scaled, y_train, cv=3)

array([0.91596639, 0.92076831, 0.92076831])

In [93]:
print(classification_report(y_train, preds_1))

              precision    recall  f1-score   support

       False       0.95      1.00      0.97      2141
        True       1.00      0.69      0.81       358

    accuracy                           0.96      2499
   macro avg       0.98      0.84      0.89      2499
weighted avg       0.96      0.96      0.95      2499



In [85]:
# preds1_test = rf1.predict(X_test_scaled)

In [87]:
# print(classification_report(y_test, preds1_test))

### Model 2 : Logistic Regression

In [47]:
logreg = LogisticRegression(random_state=42)

In [48]:
logreg.fit(X_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [49]:
preds_2 = logreg.predict(X_train_scaled)

In [31]:
print(classification_report(y_train, preds_2))

              precision    recall  f1-score   support

       False       0.87      0.99      0.92      2141
        True       0.49      0.09      0.15       358

    accuracy                           0.86      2499
   macro avg       0.68      0.54      0.53      2499
weighted avg       0.81      0.86      0.81      2499



### Model 3: KNN


In [98]:
knn = KNeighborsClassifier()
knn.fit(X_train_scaled, y_train)

preds_knn = knn.predict(X_train)

print(classification_report(y_train, preds_knn))

              precision    recall  f1-score   support

       False       0.89      0.16      0.27      2141
        True       0.15      0.88      0.25       358

    accuracy                           0.26      2499
   macro avg       0.52      0.52      0.26      2499
weighted avg       0.78      0.26      0.26      2499



### Model 4: Extra Trees